In [2]:
import datetime as dt
import pandas as pd
import numpy as np
from calendar import monthrange

In [5]:
now = datetime.datetime.now()

In [14]:
link_dat = pd.read_csv('link table 1962-2001_csv.zip',
                         parse_dates=['datadate'],
                        usecols=['datadate', 'GVKEY', 'LPERMNO', 'conm'])

In [13]:
crsp_dat = pd.read_csv('crsp returns all 1962-2001_csv.zip',
                   parse_dates=['date'],
                       usecols=['date', 'PERMNO', 'PRC', 'RET', 'SHROUT',
                                'SHRCD', 'EXCHCD', 'PRIMEXCH','DISTCD','SHRCLS'])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
compustat_dat = pd.read_csv('compustat_fundamentals qtr 1962-2001_csv.zip', header= 0 ,
                          parse_dates=['datadate'],
                          usecols=['datadate', 'gvkey', 'exchg', 'fyearq', 
                                   'datafqtr','prccq','atq','ltq','niq','fyr','rdq'])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
compustat = pd.read_csv('compustat_fundamentals qtr 1962-2001_csv.zip',
                          parse_dates=['datadate'])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10,13,17,18,20,25,26,30,33,34,35,647,648,649,650,654,656,657,664,668,669,674,675,677,679) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
crsp = pd.read_csv('crsp returns all 1962-2001_csv.zip',
                   parse_dates=['date'])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
link = pd.read_csv('link table 1962-2001_csv.zip')

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1]:
crsp.columns

NameError: name 'crsp' is not defined

In [36]:
# US listed companies
exchange_codes = {
    11: 'NYSE', 12: 'American', 14: 'Nasdaq', 
    15: 'Boston', 16: 'Chicago', 17: 'Pacific', 18: 'Philadelphia'}

compustat_dat = compustat_dat[compustat_dat.exchg.isin({1,2,3})]

# Common shares --> share class = 10 or 11
crsp_dat = crsp_dat[crsp_dat.SHRCD.isin([10, 11])]

In [37]:
import datetime
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

In [38]:
# add market cap column to crsp
crsp_dat.loc[:, 'mcap'] = np.abs(crsp_dat.SHROUT * crsp_dat.PRC)
# add the month end dates
me_date_map = dict(zip(crsp_dat.date.unique(), [last_day_of_month(pd.to_datetime(uu)) for uu in crsp_dat.date.unique()]))
crsp_dat.loc[:, 'me_date'] = [me_date_map[uu] for uu in crsp_dat.date.values]
# change column names
link_dat = link_dat.rename(columns={'GVKEY':'gvkey', 'LPERMNO':'PERMNO'})

In [39]:
merged = pd.merge(compustat_dat, link_dat, on=['gvkey', 'datadate'])
merged = merged.copy()

In [40]:
merged2 = pd.merge(merged, 
                   crsp_dat[['PERMNO', 'me_date', 'mcap','DISTCD','SHRCLS']], 
                   left_on=['PERMNO', 'datadate'], 
                   right_on=['PERMNO', 'me_date'], 
                   how='left')
merged2 = merged2.copy()

In [41]:
full_universe = merged2.copy()
full_universe = full_universe.sort_values(['gvkey', 'datafqtr'])

In [42]:
date_check = full_universe[['datadate','rdq']].dropna(axis = 0, how = 'any')

In [97]:
test_time = str(data_check.ix[3,0])[:10]

C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [132]:
full_universe.shape

(496386, 17)

In [131]:
date_check.shape

(374584, 2)

In [128]:
#date_new = date_new.strftime('%Y-%m-%d')
#type(date_check.shape[0])

0.6666666666666666


In [ ]:
n = 0
for i in date_check.index:
    date_new = str(date_check.ix[i,0])[:10]
    for j in range(0,6):
        date_new = dt.datetime.strptime(date_new, '%Y-%m-%d')+dt.timedelta(days=monthrange(int(date_new[:4]),int(date_new[5:7]))[1])
        date_new = date_new.strftime('%Y-%m-%d')
    if date_new >= date_check.ix[3,1]:
        n += 1
print(n)
print(n/date_check.shape[0])

In [21]:
def date_confidence():
    date_check = self.merged_table[['datadate','rdq']].dropna(axis = 0, how = 'any')
    n = 0
    for i in date_check.index:
        date_new = str(data_check.ix[i,0])[:10]
        for j in range(0,6):
            date_new = dt.datetime.strptime(date_new, '%Y-%m-%d')+dt.timedelta(days=monthrange(int(date_new[:4]),int(date_new[5:7]))[1])
            date_new = date_new.strftime('%Y-%m-%d')
        if date_new >= data_check.ix[3,1]:
            n += 1
    return (n/date_check.shape[0])

In [54]:
import pandas as pd
import numpy as np
import datetime as dt
from calendar import monthrange
import utils

class DataLoader:

    def __init__(self, loc):
        self.data_location = loc

        self.merged_table = None

        self.__load_data__()

        self.merge()


    def __load_data__(self):
        link_table = pd.read_csv('{loc}link table 1962-2001_csv.zip'.format(loc=self.data_location),
                                 header= 0,
                                 parse_dates=['datadate'],
                                 usecols=['datadate', 'GVKEY', 'LPERMNO', 'conm'])

        crsp = pd.read_csv('{loc}/crsp returns all 1962-2001_csv.zip'.format(loc=self.data_location),
                           header= 0,
                           parse_dates=['date'],
                           usecols=['date', 'PERMNO', 'PRC', 'RET', 'SHROUT', 'SHRCD', 'EXCHCD', 'PRIMEXCH', 'SHRCLS'])

        compustat = pd.read_csv('{loc}/compustat_fundamentals qtr 1962-2001_csv.zip'.format(loc=self.data_location),
                                header= 0 ,
                                parse_dates=['datadate'],
                                usecols=['datadate', 'gvkey', 'exchg', 'fyearq', 'datafqtr','prccq','atq','ltq','niq','rdq'])

        self.data = {'link':link_table, 'crsp':crsp, 'compustat': compustat}
        self.load_mcap_breakpoints()

        self.process_data()
        self.apply_filter()



    def apply_filter(self):
        exchg_selected = [1,2,3]
        self.data['crsp'] = self.data['crsp'][self.data['crsp'].EXCHCD.isin(exchg_selected)]

        shrcd_selected = [10,11]
        self.data['crsp']= self.data['crsp'][self.data['crsp'].SHRCD.isin(shrcd_selected)]


    def process_data(self):
        # fill empty share class with NA
        self.data['crsp']['SHRCLS'] = self.data['crsp']['SHRCLS'].fillna('NA')

        # add market cap column to crsp
        self.data['crsp'].loc[:, 'mcap'] = np.abs(self.data['crsp']['PRC']) * self.data['crsp']['SHROUT']

        # add the month end dates
        vs = [utils.last_day_of_month(pd.to_datetime(uu)) for uu in self.data['crsp'].date.unique()]
        me_date_map = dict(zip(self.data['crsp'].date.unique(), vs))
        self.data['crsp'].loc[:, 'me_date'] = [me_date_map[uu] for uu in self.data['crsp'].date.values]


    def merge(self, reload=False):
        if self.merged_table is None or reload:
            #change columns name to make sure column names are same in both database
            linktable = self.data['link'].rename(columns = {'GVKEY':'gvkey'})

            #merge compustat and link table
            tmp = pd.merge(self.data['crsp'],
                           linktable ,
                           how ='inner',
                           left_on=['PERMNO', 'me_date'],
                           right_on= ['LPERMNO', 'datadate'])

            tmp = self.aggregate_crsp_table(tmp)

            #merge with compustat
            merged = pd.merge(self.data['compustat'],
                              tmp,
                              how ='inner',
                              on=['gvkey', 'datadate'])

            self.merged_table = pd.merge(merged,
                                         self.mcap_breakpoints,
                                         how='left',
                                         left_on='datadate',
                                         right_on='mcap_date')


    def load_mcap_breakpoints(self):
        # FAMA FRENCH NYSE ME breakpoints
        ff_me_breakpoints = pd.read_csv('{loc}ME_Breakpoints_csv.zip'.format(loc=self.data_location),
                                        skiprows=[0],
                                        header=-1,
                                        names=['date', 'count'] + [5*x for x in np.arange(1,21)]).iloc[:-1]
        dts = [utils.last_day_of_month(pd.Timestamp('%s-%s-01' % (x[:4], x[4:6]))) for x in ff_me_breakpoints.date.values]
        ff_me_breakpoints.loc[:, 'date'] = dts

        self.mcap_breakpoints = ff_me_breakpoints[['date', 20]]
        self.mcap_breakpoints.columns = ['mcap_date', 'breakpt']
        self.mcap_breakpoints.loc[:, 'breakpt'] = self.mcap_breakpoints['breakpt'] * 1000  # to be consistent with the 'mcap' column


    def get_timeseries_by_column(self, col='atq'):
        raise("implement me")

    @staticmethod
    def aggregate_crsp_table(df):
        return df
    def date_confidence(self):
        date_check = self.merged_table[['datadate','rdq']].dropna(axis = 0, how = 'any')
        n = 0
        for i in date_check.index:
            date_new = str(date_check.ix[i,0])[:10]
            for j in range(0,6):
                date_new = dt.datetime.strptime(date_new, '%Y-%m-%d')+dt.timedelta(days=monthrange(int(date_new[:4]),int(date_new[5:7]))[1])
                date_new = date_new.strftime('%Y-%m-%d')
            if date_new >= date_check.ix[i,1].replace('/','-'):
                n += 1
        return (n/date_check.shape[0])

In [55]:
m = DataLoader('C:/Users/Biao Feng/source/Anaconda3/practicum/')

In [56]:
m.date_confidence()

C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:117: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:121: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


0.9994278189383518

In [57]:
date_check.ix[i,1]

C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


NameError: name 'i' is not defined

In [ ]:
date_check = self.merged_table[['datadate','rdq']].dropna(axis = 0, how = 'any')
n = 0
for i in date_check.index:
    date_new = str(date_check.ix[i,0])[:10]
            for j in range(0,6):
                date_new = dt.datetime.strptime(date_new, '%Y-%m-%d')+dt.timedelta(days=monthrange(int(date_new[:4]),int(date_new[5:7]))[1])
                date_new = date_new.strftime('%Y-%m-%d')
            if date_new >= date_check.ix[i,1]:
                n += 1
        return (n/date_check.shape[0])

In [15]:
compustat_1 = compustat.drop_duplicates()
#v = compustat_1.pivot(index = 'datadate',columns = 'gvkey',values = 'state')

In [ ]:
v = compustat_1.pivot(index = 'datadate',columns = 'gvkey',values = 'state')

In [9]:
v.count(1).plot

NameError: name 'v' is not defined

In [12]:
crsp_dat.columns

NameError: name 'crsp_dat' is not defined